<a href="https://colab.research.google.com/github/sahilvr03/Agentic-Ai-Fundamentels./blob/main/pincon_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.1 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINCON_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)


In [4]:
import time

index_name = "pincone-rag-project"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [5]:

pc = Pinecone(api_key=pinecone_api_key)

In [13]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("musab musab musab")
vector[:5]



[0.061101242899894714,
 -0.08108502626419067,
 -0.00795411691069603,
 0.01032360177487135,
 0.05072566121816635]

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [8]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
len(documents)

10

In [9]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['c6fb42e2-d390-4dd4-ad3b-bae5404889ee',
 '3d40fa59-3eef-4031-b1f0-22f6061b4287',
 '46a404f4-b116-4002-a8e5-7ab87654d565',
 'c8073901-5a5a-43b6-a1bd-d9c82e5f111e',
 '80aba0fc-d1b7-49af-ac1a-4a4d1ab8bb2f',
 'c6b57519-26b0-46ed-847e-e70e25982c1b',
 '54b4675c-b439-4cc3-a177-76bcf2f25c51',
 'e3ed3506-2fc7-4f9a-9a6c-4adcfe393247',
 '93a31f70-c4e3-42f0-80c6-6fd0649493cc',
 '71915d38-de2b-4d92-8d52-165a526a7069']

In [10]:
result = vector_store.similarity_search(
    "The top 10 soccer players in the world right now."
)

for res in result:
    print( f"*{res.page_content}[{res.metadata}]" )



*The top 10 soccer players in the world right now.[{'source': 'website'}]
*The top 10 soccer players in the world right now.[{'source': 'website'}]
*I have a bad feeling I am going to get deleted :([{'source': 'tweet'}]
*I have a bad feeling I am going to get deleted :([{'source': 'tweet'}]


In [11]:
def answer_to_user(query: str):
  #vector search
  results= vector_store.similarity_search(query, k=2)

  final_answer = model(results, query)

  return final_answer